In [5]:
# In Terminal, "pip install ibis-framework[duckdb]"
import os
import ibis
from ibis import selectors as s
from ibis import _
ibis.options.interactive = True


In [6]:
# Path
path = "../"
data_path = path + "data/datathon/"
data_path = os.path.expanduser(path + "data/datathon/")

In [7]:
data_path + "vgsales.parquet"

'../data/datathon/vgsales.parquet'

In [8]:
# Source
vgsales_original = ibis.read_csv(data_path + "vgsales.csv")
vgsales_long = ibis.read_parquet(data_path + "vgsales.parquet")

In [9]:
# Source
pricing = ibis.read_parquet(data_path + "pricing.parquet")
department = ibis.read_parquet(data_path + "department.parquet")
metrics = ibis.read_parquet(data_path + "metrics.parquet")

In [10]:
pricing.head(3)

┏━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ size       ┃ vcpu  ┃ memory  ┃ cost_per_hour ┃
┡━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string     │ int64 │ float64 │ float64       │
├────────────┼───────┼─────────┼───────────────┤
│ t3a.medium │     2 │     4.0 │        0.0376 │
│ t3.large   │     2 │     8.0 │        0.0832 │
│ t3.xlarge  │     4 │    16.0 │        0.1664 │
└────────────┴───────┴─────────┴───────────────┘

### 1.2 How many departments use the appliances of the Data Platform?

In [11]:
metrics_department = metrics.join(department, "id", how = "left").select(~s.matches("id_right"))
metrics_department.head(3)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id                  ┃ net_in  ┃ data_timestamp      ┃ net_out ┃ vcpu     ┃ disk_size ┃ created_at          ┃ updated_at          ┃ type         ┃ last_patch          ┃ size      ┃ department ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string              │ float64 │ timestamp           │ float64 │ float64  │ int64     │ timestamp           │ timestamp           │ string       │ timestamp           │ string    │ string     │
├─────────────────────┼─────────┼─────────────────────┼─────────┼──────────┼───────────┼─────────────────────┼─────────────────────┼──────────────┼─────────────────────┼───────────┼────────────┤
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ Marketing  │
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ Marketing  │
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ Marketing  │
└─────────────────────┴─────────┴─────────────────────┴─────────┴──────────┴───────────┴─────────────────────┴─────────────────────┴──────────────┴─────────────────────┴───────────┴────────────┘

In [12]:
metrics_unique = metrics_department.group_by(["id", "department", "disk_size", "type", "size"]).aggregate()
metrics_unique.head(3)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ id                  ┃ department ┃ disk_size ┃ type         ┃ size        ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string              │ string     │ int64     │ string       │ string      │
├─────────────────────┼────────────┼───────────┼──────────────┼─────────────┤
│ i-0a16592fdb1239d51 │ Marketing  │       240 │ deeplearning │ t3.xlarge   │
│ i-080c53b1618bf299a │ IT         │        90 │ rstudio      │ r5a.2xlarge │
│ i-0b464aceb0dd373fc │ Sales      │       240 │ deeplearning │ r5.2xlarge  │
└─────────────────────┴────────────┴───────────┴──────────────┴─────────────┘

In [13]:
t = metrics_unique.group_by("department").aggregate(count_department = _.department.count())
t.order_by(ibis.desc("count_department"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ department  ┃ count_department ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string      │ int64            │
├─────────────┼──────────────────┤
│ Sales       │               10 │
│ IT          │                6 │
│ HR          │                6 │
│ Engineering │                5 │
│ Marketing   │                3 │
│ Operations  │                3 │
└─────────────┴──────────────────┘

### 1.3 What is the most popular appliance size used by all departments? And how many of those popular sizes did you find in the whole dataset?

In [14]:
t = metrics_unique.group_by("size").aggregate(count_size = _.size.count())
t.order_by(ibis.desc("count_size"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ size        ┃ count_size ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string      │ int64      │
├─────────────┼────────────┤
│ t3a.medium  │         21 │
│ r5a.2xlarge │          2 │
│ t3a.2xlarge │          2 │
│ c5.4xlarge  │          2 │
│ t3.xlarge   │          1 │
│ r5.2xlarge  │          1 │
│ t3.large    │          1 │
│ g3s.xlarge  │          1 │
│ m5d.xlarge  │          1 │
│ t3a.xlarge  │          1 │
└─────────────┴────────────┘

### 2.1 Which is the most popular appliance type per department?

In [15]:
t = metrics_unique.group_by(["department", "type"]).aggregate(type_per_department=metrics_unique.type.count())
t.order_by(["department", ibis.desc("type_per_department")])

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ department  ┃ type         ┃ type_per_department ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string      │ string       │ int64               │
├─────────────┼──────────────┼─────────────────────┤
│ Engineering │ jupyter      │                   3 │
│ Engineering │ knime        │                   1 │
│ Engineering │ rstudio      │                   1 │
│ HR          │ knime        │                   5 │
│ HR          │ deeplearning │                   1 │
│ IT          │ jupyter      │                   3 │
│ IT          │ rstudio      │                   2 │
│ IT          │ knime        │                   1 │
│ Marketing   │ matlab       │                   2 │
│ Marketing   │ deeplearning │                   1 │
│ …           │ …            │                   … │
└─────────────┴──────────────┴─────────────────────┘

### 2.2 Wich appliance size had the lowest vCPU utilization over the full time range of the dataset based on the listed metrics? Calculate a value with 6 digits after zero for each metric:

In [16]:
vcpu_by_size = metrics.group_by(["id", "data_timestamp", "size", "vcpu"]).aggregate()

In [17]:
t = vcpu_by_size.group_by("size").aggregate(minimum = _.vcpu.min(),
                                            average = _.vcpu.mean(),
                                            median = _.vcpu.median(),)
t.order_by("median")

┏━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ size        ┃ minimum  ┃ average   ┃ median    ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string      │ float64  │ float64   │ float64   │
├─────────────┼──────────┼───────────┼───────────┤
│ r5a.2xlarge │ 0.000165 │  0.545604 │  0.216333 │
│ g3s.xlarge  │ 0.391572 │  0.450599 │  0.425000 │
│ t3a.2xlarge │ 0.212592 │  0.537950 │  0.480333 │
│ r5.2xlarge  │ 0.302333 │  1.507907 │  0.652333 │
│ t3.xlarge   │ 0.256002 │  2.191348 │  0.820327 │
│ t3a.xlarge  │ 0.368000 │  1.903183 │  0.970330 │
│ t3.large    │ 0.947409 │  1.075165 │  0.988330 │
│ c5.4xlarge  │ 0.105102 │  6.329230 │  0.994667 │
│ t3a.medium  │ 0.010630 │ 11.027830 │  1.443338 │
│ m5d.xlarge  │ 0.641000 │ 15.238145 │ 12.194167 │
└─────────────┴──────────┴───────────┴───────────┘

### 2.3 Which department has used the most appliances between 15.12.2022 and 16.01.2023?  How many appliances did they use in this time range?

In [18]:
t = metrics_department[(_.data_timestamp > "2022-12-15") & (_.data_timestamp < "2023-01-16")]
t = t.group_by(["id", "department"] ).aggregate()
t = t.group_by("department").aggregate(appliance_count = _.id.count())
t.order_by(ibis.desc("appliance_count"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ department  ┃ appliance_count ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string      │ int64           │
├─────────────┼─────────────────┤
│ Sales       │               6 │
│ Engineering │               4 │
│ IT          │               3 │
│ HR          │               3 │
│ Marketing   │               1 │
│ Operations  │               1 │
└─────────────┴─────────────────┘

In [19]:
t = metrics_department[(_["data_timestamp"] > "2022-12-15") & (_["data_timestamp"] < "2023-01-16")]
t = t.group_by(["id", "department"] ).aggregate()
t = t.group_by("department").aggregate(appliance_count = _["id"].count())
t.order_by(ibis.desc("appliance_count"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ department  ┃ appliance_count ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string      │ int64           │
├─────────────┼─────────────────┤
│ Sales       │               6 │
│ Engineering │               4 │
│ IT          │               3 │
│ HR          │               3 │
│ Marketing   │               1 │
│ Operations  │               1 │
└─────────────┴─────────────────┘

### 2.4 What is the most expensive size of an appliance used in the Data Platform in terms of hours used per department?

In [20]:
metrics_extended = metrics_department.join(pricing, "size", how = "left").select(~s.matches("size_right"))
metrics_extended.head(3)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ id                  ┃ net_in  ┃ data_timestamp      ┃ net_out ┃ vcpu     ┃ disk_size ┃ created_at          ┃ updated_at          ┃ type         ┃ last_patch          ┃ size      ┃ department ┃ vcpu_right ┃ memory  ┃ cost_per_hour ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string              │ float64 │ timestamp           │ float64 │ float64  │ int64     │ timestamp           │ timestamp           │ string       │ timestamp           │ string    │ string     │ int64      │ float64 │ float64       │
├─────────────────────┼─────────┼─────────────────────┼─────────┼──────────┼───────────┼─────────────────────┼─────────────────────┼──────────────┼─────────────────────┼───────────┼────────────┼────────────┼─────────┼───────────────┤
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ Marketing  │          4 │    16.0 │        0.1664 │
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ Marketing  │          4 │    16.0 │        0.1664 │
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ Marketing  │          4 │    16.0 │        0.1664 │
└─────────────────────┴─────────┴─────────────────────┴─────────┴──────────┴───────────┴─────────────────────┴─────────────────────┴──────────────┴─────────────────────┴───────────┴────────────┴────────────┴─────────┴───────────────┘

In [21]:
data_usage = metrics_extended.group_by(["department", "size", "cost_per_hour", "data_timestamp"]).aggregate()
data_usage = (data_usage
                .group_by(["department", "size", "cost_per_hour"])
                .aggregate(timestamp_count = _.data_timestamp.count()))
data_usage.order_by(ibis.desc(_.timestamp_count))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ department  ┃ size        ┃ cost_per_hour ┃ timestamp_count ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string      │ string      │ float64       │ int64           │
├─────────────┼─────────────┼───────────────┼─────────────────┤
│ Sales       │ r5.2xlarge  │        0.5040 │           16704 │
│ Sales       │ g3s.xlarge  │        0.7500 │           15864 │
│ HR          │ t3a.2xlarge │        0.3008 │           15807 │
│ Sales       │ m5d.xlarge  │        0.2260 │           14128 │
│ Engineering │ t3a.medium  │        0.0376 │           13898 │
│ IT          │ r5a.2xlarge │        0.4520 │            8737 │
│ Sales       │ t3a.medium  │        0.0376 │            3423 │
│ Marketing   │ t3.xlarge   │        0.1664 │            3282 │
│ HR          │ c5.4xlarge  │        0.6800 │            1237 │
│ Marketing   │ t3.large    │        0.0832 │            1104 │
│ …           │ …           │             … │               … │
└─────────────┴─────────────┴───────────────┴─────────────────┘

In [22]:
calc_cost = data_usage.mutate(cost = _.timestamp_count / 12 * _.cost_per_hour)
calc_cost.order_by([ibis.desc(_.department), ibis.desc(_.cost)])

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ department ┃ size        ┃ cost_per_hour ┃ timestamp_count ┃ cost       ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string     │ string      │ float64       │ int64           │ float64    │
├────────────┼─────────────┼───────────────┼─────────────────┼────────────┤
│ Sales      │ g3s.xlarge  │        0.7500 │           15864 │ 991.500000 │
│ Sales      │ r5.2xlarge  │        0.5040 │           16704 │ 701.568000 │
│ Sales      │ m5d.xlarge  │        0.2260 │           14128 │ 266.077333 │
│ Sales      │ t3a.medium  │        0.0376 │            3423 │  10.725400 │
│ Operations │ t3a.2xlarge │        0.3008 │             916 │  22.961067 │
│ Operations │ t3a.medium  │        0.0376 │              47 │   0.147267 │
│ Marketing  │ t3.xlarge   │        0.1664 │            3282 │  45.510400 │
│ Marketing  │ t3.large    │        0.0832 │            1104 │   7.654400 │
│ Marketing  │ t3a.medium  │        0.0376 │              13 │   0.040733 │
│ IT         │ r5a.2xlarge │        0.4520 │            8737 │ 329.093667 │
│ …          │ …           │             … │               … │          … │
└────────────┴─────────────┴───────────────┴─────────────────┴────────────┘

### 3.1 Which fields are important to find out if an appliance is idle - meaning that an appliance is running but no action is performed on it?  Sort the correct values in alphabetic order, before submitting your response.

### 3.2 Which appliances were idle and when?

In [23]:
r1 = metrics.select("size", "net_in", "net_out")
r2 = (r1
        .group_by("size")
        .aggregate(
                    max_net_in = _.net_in.max(),
                    max_net_out = _.net_out.max()
                    ))
max_network = r2.mutate(max_network = _.max_net_in + _.max_net_out)
max_network.head(3)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ size        ┃ max_net_in   ┃ max_net_out  ┃ max_network  ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string      │ float64      │ float64      │ float64      │
├─────────────┼──────────────┼──────────────┼──────────────┤
│ t3.xlarge   │ 1.683825e+08 │ 1.801788e+08 │ 3.485612e+08 │
│ r5a.2xlarge │ 1.960112e+09 │ 1.182566e+09 │ 3.142678e+09 │
│ r5.2xlarge  │ 3.327275e+09 │ 2.951368e+09 │ 6.278642e+09 │
└─────────────┴──────────────┴──────────────┴──────────────┘

In [24]:
r1 = (
        metrics
            .join(max_network, "size", how="left")
            .select(~s.matches("size_right"))
            .mutate(net_usage = _.net_in + _.net_out)
    )
r1.head(3)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ id                  ┃ net_in  ┃ data_timestamp      ┃ net_out ┃ vcpu     ┃ disk_size ┃ created_at          ┃ updated_at          ┃ type         ┃ last_patch          ┃ size      ┃ max_net_in   ┃ max_net_out  ┃ max_network  ┃ net_usage ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string              │ float64 │ timestamp           │ float64 │ float64  │ int64     │ timestamp           │ timestamp           │ string       │ timestamp           │ string    │ float64      │ float64      │ float64      │ float64   │
├─────────────────────┼─────────┼─────────────────────┼─────────┼──────────┼───────────┼─────────────────────┼─────────────────────┼──────────────┼─────────────────────┼───────────┼──────────────┼──────────────┼──────────────┼───────────┤
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ 1.683825e+08 │ 1.801788e+08 │ 3.485612e+08 │   55031.0 │
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ 1.683825e+08 │ 1.801788e+08 │ 3.485612e+08 │   55031.0 │
│ i-0a16592fdb1239d51 │ 30741.2 │ 2023-01-23 11:55:00 │ 24289.8 │ 0.827669 │       240 │ 2022-12-06 10:34:00 │ 2023-01-21 08:08:00 │ deeplearning │ 2023-12-01 16:22:00 │ t3.xlarge │ 1.683825e+08 │ 1.801788e+08 │ 3.485612e+08 │   55031.0 │
└─────────────────────┴─────────┴─────────────────────┴─────────┴──────────┴───────────┴─────────────────────┴─────────────────────┴──────────────┴─────────────────────┴───────────┴──────────────┴──────────────┴──────────────┴───────────┘

In [25]:
r2 = r1.mutate(
            check_network = (ibis.case()
                                .when(r1.net_usage < r1.max_network * 0.02, "idle")
                                .when(True, "running").end()),    
            check_vcpu = (ibis.case()
                            .when((r1.type == "deeplearning") & (r1.vcpu < 10),  "idle")
                            .when(r1.vcpu < 5, "idle")
                            .when(True, "running").end()),
        )
r3 = r2.mutate(
            check_idle = (ibis.case()
                            .when((r2.check_network == "idle") & (r2.check_vcpu == "idle"), "idle")
                            .when(True, "running").end())
        )
r3.filter(_.check_idle == "running").select(_.id, _.data_timestamp, _.check_network, _.check_vcpu, _.check_idle)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ id                  ┃ data_timestamp      ┃ check_network ┃ check_vcpu ┃ check_idle ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string              │ timestamp           │ string        │ string     │ string     │
├─────────────────────┼─────────────────────┼───────────────┼────────────┼────────────┤
│ i-0a16592fdb1239d51 │ 2023-01-23 11:40:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:40:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:40:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:35:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:35:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:35:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:30:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:30:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:30:00 │ running       │ idle       │ running    │
│ i-0a16592fdb1239d51 │ 2023-01-23 11:25:00 │ running       │ idle       │ running    │
│ …                   │ …                   │ …             │ …          │ …          │
└─────────────────────┴─────────────────────┴───────────────┴────────────┴────────────┘